In [34]:
import pandas as pd
import geopandas as gpd
import numpy as np

In [40]:
unique_kalpiot = pd.read_csv('unique_kalpiot_basic_google_geocoding.csv', index_col=0)
stats = gpd.read_file('data/ezorim_statistiim_2022.gdb') # taken from https://www.cbs.gov.il/he/mediarelease/Pages/2022/שכבת-אזורים-סטטיסטיים-2022.aspx
stats = stats[['SEMEL_YISHUV', 'SHEM_YISHUV', 'SHEM_YISHUV_ENGLISH','geometry']]

In [42]:
# merge statistical areas by setl code
stats.head()

,SEMEL_YISHUV,SHEM_YISHUV,SHEM_YISHUV_ENGLISH,geometry
0,7,שחר,SHAHAR,"MULTIPOLYGON (((173548.330 613473.170, 173542...."
1,10,תירוש,TIROSH,"MULTIPOLYGON (((188996.621 628298.350, 188991...."
2,11,"ניר ח""ן",NIR HEN,"MULTIPOLYGON (((173548.330 613473.170, 173355...."
3,13,חצבה,HAZEVA,"MULTIPOLYGON (((227210.799 520854.319, 227112...."
4,15,נועם,NO'AM,"MULTIPOLYGON (((179921.660 609396.480, 180280...."


In [45]:
# dissolve by Yishuv
yishuv = stats.dissolve(by='SEMEL_YISHUV').reset_index()

In [48]:
# convert to centroid
yishuv['geometry'] = yishuv['geometry'].centroid

In [51]:
# ballpark transfomration
yishuv_4326 = yishuv.to_crs(4326)

In [54]:
yishuv_4326.head()

,SEMEL_YISHUV,geometry,SHEM_YISHUV,SHEM_YISHUV_ENGLISH
0,7,POINT (34.72528 31.61955),שחר,SHAHAR
1,10,POINT (34.88656 31.75073),תירוש,TIROSH
2,11,POINT (34.71456 31.61032),"ניר ח""ן",NIR HEN
3,13,POINT (35.28064 30.77000),חצבה,HAZEVA
4,15,POINT (34.78884 31.56892),נועם,NO'AM


In [56]:
# dict yishuv_centroid
yeshuv_centroid = dict(zip(yishuv_4326['SEMEL_YISHUV'],yishuv_4326['geometry']))

In [59]:
yeshuv_centroid[7].x

34.72528461838241

In [30]:
# Missing Lat
unique_kalpiot.loc[unique_kalpiot['Latitude'].isna()].head()

,town_symbol,town_name,location_symbol,ballot_address,ballot_location,bldg_nubmer,street,geocode_address,Latitude,Longitude
1742,146,דבורה,1,דבורה,מזכירות,NaN,דבורה,"דבורה, דבורה",NaN,NaN
1781,1110,חבר,1,חבר,מזכירות,NaN,חבר,"חבר, חבר",NaN,NaN
1973,1413,מחנה יהודית,1,מחנה יהודית,מועדון חוגים,NaN,מחנה יהודית,"מחנה יהודית, מחנה יהודית",NaN,NaN
2186,526,סולם,1,סולם,"בי""ס יסודי ממלכתי",NaN,סולם,"סולם, סולם",NaN,NaN
2452,1208,הילה,1,הילה,מזכירות,NaN,הילה,"הילה, הילה",NaN,NaN


In [138]:
i = 10548
unique_kalpiot.loc[i, 'Latitude']  = 31.32432
unique_kalpiot.loc[i, 'Longitude'] = 35.02677


In [139]:
unique_kalpiot.loc[unique_kalpiot['Latitude'].isna()]

,town_symbol,town_name,location_symbol,ballot_address,ballot_location,bldg_nubmer,street,geocode_address,Latitude,Longitude


In [140]:
len(unique_kalpiot.loc[unique_kalpiot['Latitude'].isna()])

0

In [66]:
missing_town_symbols = unique_kalpiot.loc[unique_kalpiot['Latitude'].isna(), 'town_symbol'].to_list()

In [70]:
len(yishuv_4326[yishuv_4326['SEMEL_YISHUV'].isin(missing_town_symbols)])

15

In [79]:
np.nan

nan

In [75]:
yeshuv_centroid.get(1,'Nope')

'Nope'

In [83]:
unique_kalpiot['point'] = unique_kalpiot.loc[unique_kalpiot['Latitude'].isna(),'town_symbol'].apply(lambda p: yeshuv_centroid.get(p, np.nan))

In [89]:
unique_kalpiot.loc[unique_kalpiot['point'].notna(), 'Longitude'] = unique_kalpiot.loc[unique_kalpiot['point'].notna(), 'point'].apply(lambda p: p.x)
unique_kalpiot.loc[unique_kalpiot['point'].notna(), 'Latitude'] = unique_kalpiot.loc[unique_kalpiot['point'].notna(), 'point'].apply(lambda p: p.y)

In [93]:
unique_kalpiot = unique_kalpiot.drop(columns='point')

In [149]:
unique_kalpiot = unique_kalpiot.loc[unique_kalpiot['town_symbol'].notna()]
unique_kalpiot['town_symbol'] = unique_kalpiot['town_symbol'].astype(int)

/var/folders/pl/dfcs60nn16qdw7bltcc09p3h0000gn/T/ipykernel_1431/1247935057.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unique_kalpiot['town_symbol'] = unique_kalpiot['town_symbol'].astype(int)


In [153]:
unique_kalpiot['location_symbol'] = unique_kalpiot['location_symbol'].astype(int)

/var/folders/pl/dfcs60nn16qdw7bltcc09p3h0000gn/T/ipykernel_1431/3310949637.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unique_kalpiot['location_symbol'] = unique_kalpiot['location_symbol'].astype(int)


In [154]:
unique_kalpiot.to_csv('unique_kalpiot_basic_google_geocoding.csv')

In [123]:
yishuv_4326[yishuv_4326['SHEM_YISHUV'].apply(lambda p: 'עזאזמה' in p)]

,SEMEL_YISHUV,geometry,SHEM_YISHUV,SHEM_YISHUV_ENGLISH
